# Using Large Datasets
> Tutorial on how to train neuralforecast models on datasets that cannot fit into memory

The standard DataLoader class used by NeuralForecast expects the dataset to be represented by a single DataFrame, which is entirely loaded into memory when fitting the model. However, when the dataset is too large for this, we can instead use the custom large-scale DataLoader. This custom loader assumes that each timeseries is split across a collection of Parquet files, and ensure that only one timeseries is ever loaded into memory at a given time.

In this notebook, we will demonstrate the expected format of these files, how to train the model and and how to perform inference using this large-scale DataLoader.

## Load libraries

In [ ]:
import os

import numpy as np
import pandas as pd

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mae, rmse, smape
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic

## Data

Each timeseries should be stored in a directory named **unique_id=timeseries_id**. Within this directory, the timeseries can be entirely contained in a single Parquet file or split across multiple Parquet files. Regardless of the format, the timeseries must be ordered by time.

For example, the following code splits the AirPassengers DataFrame (of which each timeseries is already sorted by time) into the below format:
<br>
<br>

**\>**&nbsp;&nbsp;data  
&nbsp;&nbsp;&nbsp;&nbsp;**\>**&nbsp;&nbsp;unique_id=Airline1  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; -&nbsp;&nbsp;a59945617fdb40d1bc6caa4aadad881c-0.parquet  
&nbsp;&nbsp;&nbsp;&nbsp;**\>**&nbsp;&nbsp;unique_id=Airline2  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; -&nbsp;&nbsp;a59945617fdb40d1bc6caa4aadad881c-0.parquet  

<br>
We then simply input a list of the paths to these directories.

In [ ]:
Y_df = AirPassengersPanel.copy()
Y_df

,unique_id,ds,y,trend,y_[lag12]
0,Airline1,1949-01-31,112.0,0,112.0
1,Airline1,1949-02-28,118.0,1,118.0
2,Airline1,1949-03-31,132.0,2,132.0
3,Airline1,1949-04-30,129.0,3,129.0
4,Airline1,1949-05-31,121.0,4,121.0
...,...,...,...,...,...
283,Airline2,1960-08-31,906.0,283,859.0
284,Airline2,1960-09-30,808.0,284,763.0
285,Airline2,1960-10-31,761.0,285,707.0
286,Airline2,1960-11-30,690.0,286,662.0


In [ ]:
valid = Y_df.groupby('unique_id').tail(72)
# from now on we will use the id_col as the unique identifier for the timeseries (this is because we are using the unique_id column to partition the data into parquet files)
valid = valid.rename(columns={'unique_id': 'id_col'})

train = Y_df.drop(valid.index)
train['id_col'] = train['unique_id'].copy()
train.to_parquet('./data', partition_cols=['unique_id'], index=False)

files_list = [f"data/{dir}" for dir in os.listdir('./data')]
files_list

['data/unique_id=Airline2', 'data/unique_id=Airline1']

The DataLoader class still expects the static data to be passed in as a single DataFrame with one row per timeseries.

In [ ]:
static = AirPassengersStatic.rename(columns={'unique_id': 'id_col'})
static

,id_col,airline1,airline2
0,Airline1,0,1
1,Airline2,1,0


## Model training

We now train a NHITS model on the above dataset. 
It is worth noting that NeuralForecast currently does not support scaling when using this DataLoader. If you want to scale the timeseries this should be done before passing it in to the `fit` method.

In [ ]:
horizon = 12
stacks = 3
models = [NHITS(input_size=5 * horizon,
                h=horizon,
                futr_exog_list=['trend', 'y_[lag12]'],
                stat_exog_list=['airline1', 'airline2'],
                max_steps=100,
                stack_types = stacks*['identity'],
                n_blocks = stacks*[1],
                mlp_units = [[256,256] for _ in range(stacks)],
                n_pool_kernel_size = stacks*[1])]
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=files_list, static_df=static, id_col='id_col')

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type          | Params
-----------------------------------------------
0 | loss         | MAE           | 0     
1 | padder_train | ConstantPad1d | 0     
2 | scaler       | TemporalNorm  | 0     
3 | blocks       | ModuleList    | 802 K 
-----------------------------------------------
802 K     Trainable params
0         Non-trainable params
802 K     Total params
3.211     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.


## Forecasting

When working with large datasets, we need to provide a single DataFrame containing the input timesteps of all the timeseries for which wish to generate predictions. If we have future exogenous features, we should also include the future values of these features in the separate `futr_df` DataFrame. 

For the below prediction we are assuming we only want to predict the next 12 timesteps for Airline2.

In [ ]:
valid_df = valid[valid['id_col'] == 'Airline2']
# we set input_size=60 and horizon=12 when fitting the model
pred_df = valid_df[:60]
futr_df = valid_df[60:72]
futr_df = futr_df.drop(["y"], axis=1)

predictions = nf.predict(df=pred_df, futr_df=futr_df, static_df=static).reset_index()

/home/jasminerienecker/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/utilsforecast/processing.py:382: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/jasminerienecker/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/utilsforecast/processing.py:436: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

/home/jasminerienecker/Desktop/neuralforecast/neuralforecast/core.py:203: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [ ]:
predictions

,id_col,ds,NHITS
0,Airline2,1960-01-31,705.761780
1,Airline2,1960-02-29,735.606262
2,Airline2,1960-03-31,744.876282
3,Airline2,1960-04-30,778.747192
4,Airline2,1960-05-31,822.874390
5,Airline2,1960-06-30,859.136719
6,Airline2,1960-07-31,910.383240
7,Airline2,1960-08-31,890.466064
8,Airline2,1960-09-30,849.356995
9,Airline2,1960-10-31,813.761475


## Evaluation

In [ ]:
target = valid_df[60:72]

In [ ]:

(
    evaluate(
        predictions.merge(target.drop(["trend", "y_[lag12]"], axis=1), on=['id_col', 'ds']),
        metrics=[mae, rmse, smape],
        id_col='id_col',
    )
    .drop(columns='id_col')
    .groupby('metric')
    .mean()
)

,NHITS
metric,
mae,31.430094
rmse,37.789986
smape,0.020317
